<a href="https://colab.research.google.com/github/olonok69/Anomaly_detection/blob/master/spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

!pip install -q spacy==2.3.2 
!python -m spacy download en_core_web_lg
!pip install -q nltk

In [ ]:
import pandas as pd
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
#spacy_stopwords.remove('a')
import re
import datetime
from spacy.matcher import Matcher
from spacy.util import filter_spans

In [ ]:
pattern = [{'POS': 'VERB', 'OP': '?'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'AUX', 'OP': '*'},
           {'POS': 'VERB', 'OP': '+'}]

# instantiate a Matcher instance
matcher = Matcher(nlp.vocab)
matcher.add("Verb phrase", None, pattern)

In [ ]:
datap= pd.read_csv("/content/drive/My Drive/datasets/defects.csv")

In [ ]:
datap['keywords']=datap['correct_Action']= datap['spacy_verb']=datap['actions']=datap['verb_Adj']=""

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS
from collections import OrderedDict
import numpy as np
import datetime

In [ ]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        listw=[]
        for i, (key, value) in enumerate(node_weight.items()):
            #print(key + ' - ' + str(value))
            listw.append(key)
            if i > number:
                break
        return listw
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [ ]:
tr4w = TextRank4Keyword()

In [ ]:
lverbs=['VBD','VBP','VBZ', 'VB']
lverbs2=[ 'VBG','VBN']

def apply_get_actions(text, lverbs):
    
    tokens = word_tokenize(text)
    out=[]
    for i in range(len(tokens)):
        if pos_tag(tokens)[i][1] in lverbs:
            out.append(pos_tag(tokens)[i][0])
    return out



def apply_get_keywords(text, tr4w):
    
    tr4w.analyze(text.lower(), candidate_pos = ['PROPN',"NOUN"], window_size=5, lower=True)
    out=tr4w.get_keywords(3)
    if len(out)>3:
        return out[:3]
    else:
        return out
def correct_mispelled_t(text):
  
  text1=[x for x in text.split() if not(x in spacy_stopwords)]
  text2=[x for x in text.split() if not(x in spacy_stopwords)]
  for w1, w2 in zip(text1[:-1],text2[1:]):
    word = Word(w1+" "+w2)
    out=word.spellcheck()
    if out[0][1]==1.0:
      text = re.sub(word, out[0][0], text)
  return text
def remove_jpg(text):
  
  text = re.sub(r'jpg','',text)
  return text
def remove_punkt(text):
  
  text = re.sub(r'[^a-zA-Z\s\-\']',' ',text)
  return text
def spacy_action_description(texto, matcher, nlp):
  #texto=row.correct_Action_clean
  doc = nlp(texto)
  # call the matcher to find matches 
  matches = matcher(doc)
  spans = [doc[start:end] for _, start, end in matches]
  out =filter_spans(spans)
  out2=[s.text for s in out ]
  return out2

In [ ]:
# correct mispelled words on text
# Iterate on the index as is 20 times faster than apply the fucntion
tim1= datetime.datetime.now()
for i in datap.index:
 
    empty_list=[]
    texto= datap.at[i,'Action_Comment']
    # fixing mispelled words
    try:
      
      texto_clean=correct_mispelled_t(texto)
    except:
      texto_clean=texto
    
    # remove numbers and and punctuation signs and translate with spaces
    try:
      texto_clean=remove_jpg(texto_clean)
      only_words= remove_punkt(texto_clean)
    except:
      only_words = texto_clean    
    
    datap.at[i,'correct_Action']=only_words 
    
    # find keywords
    try:
      datap.at[i,'keywords']=apply_get_keywords(only_words, tr4w)
    except:
      datap.at[i,'keywords']=empty_list
    
    # find verbs / actions using spacy

    # texto_clean=datap.at[i,'correct_Action'] 
    # try:
    #   datap.at[i,'spacy_verb']=spacy_action_description(texto_clean,matcher,nlp)
    # except:
    #   datap.at[i,'spacy_verb']=empty_list
    
    # # find verbs using nltk
    # try:
    #   datap.at[i,'actions']=apply_get_actions(texto_clean,lverbs)
    # except:
    #   datap.at[i,'actions']=empty_list
    # try:
    #   datap.at[i,'verb_Adj']=apply_get_actions(texto_clean,lverbs2)
    # except:
    #   datap.at[i,'verb_Adj']=empty_list
    
    
    if i%5000==0:
      print (i)


tim2= datetime.datetime.now()        
print(tim2-tim1)

In [ ]:
datap.to_csv("/content/drive/My Drive/datasets/defects_2.csv", index=False)

In [ ]:
datap= pd.read_csv("/content/drive/My Drive/datasets/defects_2.csv")

In [ ]:
datap.head(100)

In [ ]:
datap.Element_Name.unique()

In [ ]:
datap.Feature_Name.unique()